In [1]:
import cv2
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

def extract_frames(video_path, max_frames=50):
    cap = cv2.VideoCapture(video_path)
    frames = []
    count = 0
    while cap.isOpened() and count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        resized_frame = cv2.resize(gray_frame, (128, 128))
        frames.append(resized_frame)
        count += 1
    cap.release()
    return frames

def apply_gabor(image):
    features = []
    for theta in (0, np.pi/4, np.pi/2, 3*np.pi/4):  # 4 orientations
        kernel = cv2.getGaborKernel((21, 21), 5.0, theta, 10.0, 0.5, 0, ktype=cv2.CV_32F)
        filtered = cv2.filter2D(image, cv2.CV_8UC3, kernel)
        features.append(np.mean(filtered))
        features.append(np.std(filtered))
    return np.array(features)

def load_dataset_gabor(video_paths, labels, max_frames=50):
    features = []
    new_labels = []
    for i, video_path in enumerate(video_paths):
        if not os.path.exists(video_path):
            print(f"Error: {video_path} not found.")
            continue
        frames = extract_frames(video_path, max_frames)
        if len(frames) == 0:
            print(f"Warning: No frames in {video_path}")
            continue
        for frame in frames:
            gabor_feat = apply_gabor(frame)
            features.append(gabor_feat)
            new_labels.append(labels[i])
    return np.array(features), np.array(new_labels)

def train_random_forest(features, labels):
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, stratify=labels, random_state=42)
    clf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"✅ Accuracy with Gabor Features: {accuracy:.4f}")
    return clf

def prepare_ffpp_dataset(base_path):
    video_paths = []
    labels = []
    for label, folder in enumerate(['real', 'fake']):  # 0 = real, 1 = fake
        folder_path = os.path.join(base_path, folder)
        if not os.path.exists(folder_path):
            print(f"Warning: Folder {folder_path} not found.")
            continue
        for video_file in os.listdir(folder_path):
            if video_file.endswith('.mp4'):
                video_paths.append(os.path.join(folder_path, video_file))
                labels.append(label)
    return video_paths, labels


base_path = "C:\\Users\\Pavan\\Downloads\\FF++" 
video_paths, labels = prepare_ffpp_dataset(base_path)

try:
    features, labels = load_dataset_gabor(video_paths, labels)
    if len(features) > 0:
        model = train_random_forest(features, labels)
    else:
        print("❌ No features extracted. Check your dataset paths and content.")
except Exception as e:
    print(f"❌ Error: {e}")

✅ Accuracy with Gabor Features: 0.8023
